In [1]:
import os
import random

In [2]:
archivo = random.sample(os.listdir("C:\Users\HP-Laptop\Desktop\INTELIGENCIA 2\PARCIAL 3\CORPUS"),1)
archivo = archivo[0]

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (Temp/ipykernel_20224/3504888549.py, line 1)

In [ ]:
with open("D:/CORPUS/"+archivo,"r", encoding="utf8") as entrada:
    texto = entrada.read()

In [ ]:
texto.split()

In [ ]:
separadores = ["(",")",",",".",";",":","\"","¿","?","¡","!","--","_"]

for separador in separadores:
    texto = texto.replace(separador," ")

In [ ]:
texto.split()

In [ ]:
#!pip install nltk
# Se debe reiniciar el kernel después de la instalación

In [ ]:
import os
import random
import nltk

In [ ]:
nltk.download("punkt")

In [ ]:
tokens = nltk.word_tokenize(texto,"spanish")
tokens

In [ ]:
# es importante revisar el contexto de nuestro corpus, porque en ocasiones es neecsario modificar algunas características
# del tokenizador. Por ejemplo: aquí quitamos los guiones al inicio y al final.

for i,token in enumerate(tokens):
    if token.startswith("_") or token.startswith("—"):
        tokens[i] = tokens[i][1:]
    if token.endswith("_") or token.endswith("—"):
        tokens[i] = tokens[i][:-1]
texto = " ".join(tokens)

In [ ]:
tokens = nltk.word_tokenize(texto,"spanish")
tokens

In [ ]:
# def tokenizar(texto):
#    puntuacion = ''
 #   tokens = nltk.word_tokenize(texto,"spanish")
  #  for i,token in enumerate(tokens):
        if token.startswith("_") or token.startswith("—"):
            tokens[i] = tokens[i][1:]
        if token.endswith("_") or token.endswith("—"):
            tokens[i] = tokens[i][:-1]
    texto = " ".join(tokens)
    tokens = nltk.word_tokenize(texto,"spanish")
    return tokens

In [ ]:
def tokenizar(texto):
    puntuacion = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¿¡'
    tokens = nltk.word_tokenize(texto,"spanish")
    for i,token in enumerate(tokens):
        tokens[i] = token.strip(puntuacion)
    texto = " ".join(tokens)
    tokens = nltk.word_tokenize(texto,"spanish")
    return tokens

In [ ]:
indice = {}
corpus = os.listdir("D:/CORPUS")
for archivo in corpus:
    with open("D:/CORPUS/"+archivo,"r", encoding="utf8") as entrada:
        texto = entrada.read()
    tokens = tokenizar(texto)
    vocabulario = set(tokens)
    for palabra in vocabulario:
        if palabra not in indice:
            indice[palabra] = set()
        indice[palabra].add(archivo)

In [ ]:
indice["México"]

In [ ]:
indice["México"].intersection(indice["tecnología"])

In [ ]:
indice[""]

In [ ]:
len(indice)

In [ ]:
indice

In [ ]:
def stemmizar(tokens):
    stemmer = nltk.stem.SnowballStemmer("spanish")
    stems = []
    for token in tokens:
        stem = stemmer.stem(token)
        stems.append(stem)
    return stems

In [ ]:
# hacemos un nuevo índice
indice = {}
corpus = os.listdir("D:/CORPUS")
for archivo in corpus:
    with open("D:/CORPUS/"+archivo,"r", encoding="utf8") as entrada:
        texto = entrada.read()
        texto = texto.lower()
    tokens = tokenizar(texto)
    stems = stemmizar(tokens)
    vocabulario = set(stems)
    for entrada in vocabulario:
        if entrada not in indice:
            indice[entrada] = set()
        indice[entrada].add(archivo)

In [ ]:
# a partir de aquí las palabras a buscar deben de ir con la función stemmizar
indice[stemmizar(["oreja"])[0]]
#  así no: indice["oreja"]

In [ ]:
len(indice)

In [ ]:
# importamos pandas
import pandas as pd

In [ ]:
indice_frecuencias = {}
corpus = os.listdir("D:/CORPUS")
for archivo in corpus:
    with open("D:/CORPUS/"+archivo,"r", encoding="utf8") as entrada:
        texto = entrada.read()
        texto = texto.lower()
    tokens = tokenizar(texto)
    stems = stemmizar(tokens)
    vocabulario = stems
    for entrada in vocabulario:
        if entrada not in indice_frecuencias:
            indice_frecuencias[entrada] = {}
        if archivo not in indice_frecuencias[entrada]: 
            indice_frecuencias[entrada][archivo]=0
        indice_frecuencias[entrada][archivo]+=1

In [ ]:
vocabulario = indice_frecuencias.keys()
corpus = corpus # solo como recordatorio de que alli estan los archivos
tabla_frecuencias = pd.DataFrame(0,index=vocabulario,columns=corpus)
for entrada in indice_frecuencias:
    for archivo in indice_frecuencias[entrada]:
        tabla_frecuencias.loc[entrada,archivo] = indice_frecuencias[entrada][archivo]

In [ ]:
tabla_frecuencias

In [ ]:
archivo = random.sample(os.listdir("d:/CORPUS"),1)
archivo = archivo[0]
tabla_frecuencias[archivo].sort_values(ascending=False).head(20)

In [ ]:
nltk.download("stopwords")

palabras_funcionales=nltk.corpus.stopwords.words("spanish")
print(palabras_funcionales)

In [ ]:
def tokenizador(texto):
    tokens = tokenizar(texto)
    stems = stemmizar(tokens)
    return stems

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
lista_archivos = ["D:/CORPUS/"+archivo for archivo in corpus]
vectorizador = CountVectorizer(input="filename",analyzer="word")
#vectorizador = CountVectorizer(input="filename",analyzer="word",tokenizer=tokenizador)
#vectorizador = CountVectorizer(input="filename",analyzer="word",tokenizer = tokenizador, stop_words=palabras_funcionales)
matriz_frecuencias = vectorizador.fit_transform(lista_archivos)

In [ ]:
matriz_frecuencias.toarray()

In [ ]:
vocabulario = vectorizador.get_feature_names()
corpus = corpus # solo como recordatorio de que alli estan los archivos
tabla_frecuencias = pd.DataFrame(matriz_frecuencias.toarray(),index=corpus,columns=vocabulario)
tabla_frecuencias

In [ ]:
archivo = random.sample(os.listdir("D:/CORPUS"),1)
archivo = archivo[0]
tabla_frecuencias.loc[archivo].sort_values(ascending=False).head(20)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
corpus = os.listdir("D:/CORPUS")
lista_archivos = ["D:/CORPUS/"+archivo for archivo in corpus]
vectorizador = TfidfVectorizer(input="filename",analyzer="word",sublinear_tf=True)
matriz_tfidf = vectorizador.fit_transform(lista_archivos)
vocabulario = vectorizador.get_feature_names()
tabla_tfidf = pd.DataFrame(matriz_tfidf.toarray(),index=corpus,columns=vocabulario)
tabla_tfidf

In [ ]:
archivo = random.sample(os.listdir("D:/CORPUS"),1)
archivo = archivo[0]
tabla_tfidf.loc[archivo].sort_values(ascending=False).head(20)

In [ ]:
tabla_tfidf[["méxico","tecnología"]].sum(axis=1).sort_values(ascending=False).head(20)